In [1]:
import pandas as pd
import datetime as dt
from flask import Flask
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:

# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Carlos\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
mars_news = 'https://redplanetscience.com/'
browser.visit(mars_news)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Retrieve the News Title
news_title = soup.body.find('div', class_='content_title')
news_title.text

"NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes"

In [6]:

# Retrieve the paragraph text
news_p = soup.find('div', class_='article_teaser_body')
news_p.text

'After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.'

In [7]:
mars_images = 'https://spaceimages-mars.com/'
browser.visit(mars_images)

In [8]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [9]:

image_url = soup.find('img', class_='headerimage')['src']
featured_image_url = f'https://spaceimages-mars.com/{image_url}'
featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [10]:
#defining target url
mars_facts =  'https://galaxyfacts-mars.com/'

In [11]:

# use pandas to read in html from previously defined url
facts = pd.read_html(mars_facts)

# grabbing the second table with only Mars facts
df = facts[0]
df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [12]:
#renaming columns
df.rename(columns={0:'Description', 1:'Mars', 2:'Earth'}, inplace=True)

In [13]:

df.set_index('Description', drop=True, inplace=True)

In [14]:
df.drop(['Mars - Earth Comparison'], axis=0, inplace=True)
df

,Mars,Earth
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
#converting df to html table for later use
df.to_html('mars_facts_table.html')

In [16]:
#saving table to variable for app
mars_table = df.to_html().replace('\n', '')
mars_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Description</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [17]:
url = 'https://marshemispheres.com/'
browser.visit(url)

In [18]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [19]:

hemisphere_image_urls = []
results = soup.find_all('div', class_='item')
for result in results:
    images = {}
    title = result.find('h3').text
    images['title'] = title[:-9]
    browser.find_by_text(title).click()
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    images['url'] = url + soup.find('img', class_='wide-image')['src']
    browser.back()
    hemisphere_image_urls.append(images)

In [20]:

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [21]:

browser.quit()